# Setting up a simple authentication

In this section we will set up a simple PAM configuration to allow certificate authentication for using the yubikey. 

_The trend currently is to start to use `sssd`, unfortunately not all distrobutions will offer the 2.x version of SSSD that allows for local certificate authentication (Debian buster provide sssd 1.16 version). For that reason we will use the older (no longer maintained) project `pam_pkcs11`. I believe the use of `yubico-pam` module could also do similar functionality, but for our purposes we will use the `pam_pkcs11` module which I am more familiar with._

In Debian the `/etc/pam_pkcs11/pam_pkcs11.conf` is not part of the default load, using the example file to create the configuration can be found `/usr/share/doc/libpam-pkcs11/examples/pam_pkcs11.conf.example.gz`. Debian has also built the libpam-pkcs11 module using OpenSSL, so this method will be using the openssl hash links to verify the certificates. 

First let's make the user for testing.

In [ ]:
%%bash
sudo adduser --gecos "Rachel Andomsoul" --disabled-password randomsoul

Now we can set up the `pam_pkcs11.conf` file and select it for using the opensc section. 

Some important configuration details: 

```
use_pkcs11_module = opensc;

pkcs11_module opensc {
  module = /usr/lib/x86_64-linux-gnu/opensc-pkcs11.so;
  description = "OpenSC PKCS#11 module";
  ca_dir = /etc/pam_pkcs11/cacerts;
  crl_dir = /etc/pam_pkcs11/crls;
  cert_policy = ca, signature;
}

use_mappers = generic, null;

mapper generic {
  debug = true;
  module = internal;
  ignorecase = false;
  cert_item  = cn;
  mapfile = file:///etc/pam_pkcs11/generic_mapping;
  use_getpwent = false;
}
```

The choice in using generic because of some issues found with cn and uid internal mappers not always working, likely it is something simple. However, the use of the generic mapping mechanism does transition well to the `sssd` local certificate method in which use of a regex style subject mapping to local user is use instead of the internal mapping mechanisms like cn and uid.

The `/etc/pam_pkcs11/generic_mapping` file contains a line for the test user: 

```
Rachel Andomsoul -> randomsoul
```

This configuration will use the openssl style certificates with hash reference to provide the authentication matching.

We need to copy over the `ca-root.pem` file into the cacerts location designated in the file. We then need to run the openssl rehash function.

In [ ]:
%%bash
sudo cp certificates/ca-root.pem /etc/pam_pkcs11/cacerts
sudo openssl rehash /etc/pam_pkcs11/cacerts

We should find that the root certificate and its hash link are available.

In [ ]:
%%bash
ls -l /etc/pam_pkcs11/cacerts

In the `/etc/pam.d/common-auth` file we will add the following: 

```
auth  [success=ok default=2]                    pam_succeed_if.so user ingroup randomsoul
auth  [success=done ignore=ignore default=die]  pam_pkcs11.so debug wait_for_card
auth  [success=1 default=ignore]                pam_unix.so 
auth  requisite                                 pam_deny.so
auth  required                                  pam_permit.so
```

_This PAM config is going to force our test user to only be allowed to use the pam_pkcs11 login, while all other users can use the regular passwords, that way we can test out our configuration._

We can use the VM to show a demonstration of the card authenticating.

__DEMONSTRATE__

Now we can show a failure to log in with smartcard by removing the cacerts.

In [ ]:
%%bash
sudo rm -f /etc/pam_pkcs11/cacerts/*

__DEMONSTATE__

## A note on using NSS3 and pam_pkcs11

In the case of using Red Hat the `pam_pkcs11` module is built with NSS3 instead of OpenSSL, for this reason the following configuration would be used instead. 

```
sudo certutil -d /etc/pki/nssdb -A -n "Root Identity Certificate" -t "C,C,C" -i certificates/ca-root.pem
sudo modutil -dbdir /etc/pki/nssdb -add "OpenSC PKCS#11 Module" -libfile /usr/lib64/pkcs11/opensc-pkcs11.so -force
```

In the `pam_pkcs11.conf` file to look change the following: 

```
pkcs11_module opensc {
  module = /usr/lib/x86_64-linux-gnu/opensc-pkcs11.so;
  description = "OpenSC PKCS#11 module";
  ca_dir = /etc/pam_pkcs11/cacerts;
  crl_dir = /etc/pam_pkcs11/crls;
  cert_policy = ca, signature;
  # adding the nssdb 
  nss_dir = "/etc/pki/nssdb";
}
```

The `generic_mapping` remains the same.

Switching using the symobolic links.